In [1]:
import os
import warnings

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.options.mode.chained_assignment = None

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 10.0)
import seaborn as sns

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

Data source: https://data.colorado.gov/Demographics/Census-Counties-in-Colorado-2017/ewkj-ipn7

In [2]:
demog_2017 = pd.read_csv('../../npsg_datafiles/Census_Counties_in_Colorado_2017.csv', encoding = 'latin-1') 

In [3]:
demog_2017 = demog_2017.drop(['the_geom', 'geonum'], axis=1)
demog_2017 = demog_2017.rename(columns = {'geoname':'county'})
demog_2017.columns = [col_name + '_2017' for col_name in demog_2017.columns]

Free and reduced lunch data for Pre-K through 12th grade students for the 2016 - 2017 school year.

Free and reduced lunch data is compiled in October of each year (source: see below).

Data for the 2011-2012 school year is available in a PDF file. So far, I have not found another source for the year 2012.

Data source: https://www.cde.state.co.us/cdereval/pupilcurrentdistrict

In [4]:
free_reduced_tmp = pd.read_excel('../../npsg_datafiles/2016-17_PK_12FreeandReducedLunchEligibilitybyCounty_edited.xlsx', encoding = 'latin-1') 

In [5]:
free_reduced_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 8 columns):
COUNTY CODE           253 non-null object
COUNTY NAME           252 non-null object
DISTRICT CODE         253 non-null object
DISTRICT NAME         186 non-null object
PK-12 COUNT           253 non-null float64
NOT ELIGIBLE          248 non-null float64
FREE AND REDUCED      248 non-null float64
% FREE AND REDUCED    248 non-null float64
dtypes: float64(4), object(4)
memory usage: 16.0+ KB


In [6]:
free_reduced_tmp.tail(10)

,COUNTY CODE,COUNTY NAME,DISTRICT CODE,DISTRICT NAME,PK-12 COUNT,NOT ELIGIBLE,FREE AND REDUCED,% FREE AND REDUCED
244,90,COLORADO BOCS,9130,EXPEDITIONARY BOCES,393.0,367.0,26.0,0.066158
245,90,COLORADO BOCS,9170,COLORADO DIGITAL BOCES,2136.0,1546.0,590.0,0.276217
246,90,COLORADO BOCS,TOTAL,NaN,2897.0,2211.0,686.0,0.236797
247,98,NONE,8001,CHARTER SCHOOL INSTITUTE,16427.0,9183.0,7244.0,0.440981
248,98,NONE,9000,COLORADO SCHOOL FOR THE DEAF AND BLIND,203.0,74.0,129.0,0.635468
249,98,NONE,TOTAL,NaN,16630.0,9257.0,7373.0,0.443355
250,999,No County,9999,COLORADO DETENTION CENTER TOTAL,193.0,153.0,40.0,0.207254
251,999,No County,TOTAL,NaN,193.0,153.0,40.0,0.207254
252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,STATE,NaN,TOTAL,NaN,905019.0,523726.0,381103.0,0.421099


In [7]:
free_reduced_tmp = free_reduced_tmp.loc[free_reduced_tmp['DISTRICT CODE'] == 'TOTAL']

In [8]:
free_reduced_tmp = free_reduced_tmp.loc[free_reduced_tmp['COUNTY CODE'] <= '90']

In [9]:
free_reduced_tmp = free_reduced_tmp.loc[free_reduced_tmp['COUNTY CODE'] != 'STATE']
free_reduced_2017 = free_reduced_tmp.loc[free_reduced_tmp['COUNTY CODE'] != 'NaN']

In [10]:
free_reduced_2017.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
free_reduced_2017.rename(columns=lambda x: x.lower(), inplace=True)
free_reduced_2017.rename(columns=lambda x: x.replace('%', 'pcnt'), inplace=True)
free_reduced_2017 = free_reduced_2017.drop(['district_code', 'district_name'], axis=1)

In [11]:
free_reduced_2017.columns = [col_name + '_2017' for col_name in free_reduced_2017.columns]

In [12]:
free_reduced_2017['county_name_2017'] = free_reduced_2017['county_name_2017'].str.title()

In [13]:
free_reduced_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 7 to 246
Data columns (total 6 columns):
county_code_2017              64 non-null object
county_name_2017              64 non-null object
pk-12_count_2017              64 non-null float64
not_eligible_2017             64 non-null float64
free_and_reduced_2017         64 non-null float64
pcnt_free_and_reduced_2017    64 non-null float64
dtypes: float64(4), object(2)
memory usage: 3.5+ KB


Data source: https://data.colorado.gov/Demographics/Census-Counties-in-Colorado-2012/xwky-bmsn

In [14]:
demog_2012 = pd.read_csv('../../npsg_datafiles/Census_Counties_in_Colorado_2012.csv', encoding = 'latin-1') 

In [15]:
demog_2012 = demog_2012.drop(['the_geom', 'geonum'], axis=1)
demog_2012 = demog_2012.rename(columns = {'geoname':'county'})
demog_2012.columns = [col_name + '_2012' for col_name in demog_2012.columns]

Projections by gender and age 1990 - 2040
Data source: https://data.colorado.gov/Demographics/Population-Projections-in-Colorado/q5vp-adf3

In [16]:
demog_proj = pd.read_csv('../../npsg_datafiles/Population_Projections_in_Colorado.csv', encoding = 'latin-1') 

In [17]:
demog_proj = demog_proj.drop('dataType', axis=1)

In [18]:
demog_proj_2025 = demog_proj.loc[demog_proj['year'] == 2025]

In [19]:
cols = ['malePopulation', 'femalePopulation', 'totalPopulation']
demog_proj_2025[cols] = demog_proj_2025[cols].apply(pd.to_numeric, errors='coerce')

In [20]:
demog_proj_2025 = demog_proj_2025.round(decimals=0)

In [21]:
demog_proj_2025.fillna(0)

,county,fipsCode,year,age,malePopulation,femalePopulation,totalPopulation
123345,Archuleta,7,2025,47,92.0,104.0,196.0
123399,Douglas,35,2025,84,448.0,526.0,974.0
123437,Kiowa,61,2025,76,7.0,8.0,16.0
123532,Eagle,37,2025,92,8.0,12.0,21.0
123573,Moffat,81,2025,64,75.0,85.0,160.0
123606,Archuleta,7,2025,49,86.0,85.0,170.0
123644,La Plata,67,2025,35,274.0,246.0,520.0
123654,Cheyenne,17,2025,10,16.0,15.0,31.0
123665,Crowley,25,2025,23,62.0,22.0,84.0
123705,Jackson,57,2025,44,8.0,8.0,15.0


In [22]:
demog_proj_2025.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6464 entries, 123345 to 381497
Data columns (total 7 columns):
county              6464 non-null object
fipsCode            6464 non-null int64
year                6464 non-null int64
age                 6464 non-null int64
malePopulation      5645 non-null float64
femalePopulation    5621 non-null float64
totalPopulation     5426 non-null float64
dtypes: float64(3), int64(3), object(1)
memory usage: 404.0+ KB


In [23]:
print(demog_proj_2025.loc[demog_proj_2025['age'] <= 5])

             county  fipsCode  year  age  malePopulation  femalePopulation  \
123982       Teller       119  2025    3           155.0             148.0   
124271   Rio Blanco       103  2025    4            42.0              40.0   
125173       Pueblo       101  2025    3           980.0             938.0   
125442   Las Animas        71  2025    2            66.0              64.0   
125605     Huerfano        55  2025    2            28.0              27.0   
127712     Arapahoe         5  2025    0             NaN               NaN   
128530      Prowers        99  2025    0            75.0              72.0   
128595   Las Animas        71  2025    0            63.0              60.0   
128992     Costilla        23  2025    4            19.0              18.0   
129417        Routt       107  2025    2           165.0             158.0   
129794         Yuma       125  2025    0            75.0              72.0   
130130      Alamosa         3  2025    4           112.0        

In [24]:
demog_proj_2025['age85_pl'] = np.where(demog_proj_2025['age'] >= 85, demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age80_84'] = np.where((demog_proj_2025['age'] >= 80) & (demog_proj_2025['age'] <= 84), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age75_79'] = np.where((demog_proj_2025['age'] >= 75) & (demog_proj_2025['age'] <= 79), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age70_74'] = np.where((demog_proj_2025['age'] >= 70) & (demog_proj_2025['age'] <= 74), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age65_69'] = np.where((demog_proj_2025['age'] >= 65) & (demog_proj_2025['age'] <= 69), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age60_64'] = np.where((demog_proj_2025['age'] >= 60) & (demog_proj_2025['age'] <= 64), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age55_59'] = np.where((demog_proj_2025['age'] >= 55) & (demog_proj_2025['age'] <= 59), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age50_54'] = np.where((demog_proj_2025['age'] >= 50) & (demog_proj_2025['age'] <= 54), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age45_49'] = np.where((demog_proj_2025['age'] >= 45) & (demog_proj_2025['age'] <= 49), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age40_44'] = np.where((demog_proj_2025['age'] >= 40) & (demog_proj_2025['age'] <= 44), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age35_39'] = np.where((demog_proj_2025['age'] >= 35) & (demog_proj_2025['age'] <= 39), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age30_34'] = np.where((demog_proj_2025['age'] >= 30) & (demog_proj_2025['age'] <= 34), demog_proj_2025['totalPopulation'], 0)

In [25]:
demog_proj_2025['age25_29'] = np.where((demog_proj_2025['age'] >= 25) & (demog_proj_2025['age'] <= 29), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age20_24'] = np.where((demog_proj_2025['age'] >= 20) & (demog_proj_2025['age'] <= 24), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age15_19'] = np.where((demog_proj_2025['age'] >= 15) & (demog_proj_2025['age'] <= 19), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age10_14'] = np.where((demog_proj_2025['age'] >= 10) & (demog_proj_2025['age'] <= 14), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age5_9'] = np.where((demog_proj_2025['age'] >= 5) & (demog_proj_2025['age'] <= 9), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['ageless_5'] = np.where((demog_proj_2025['age'] <= 5), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['ageless_18'] = np.where((demog_proj_2025['age'] <= 18), demog_proj_2025['totalPopulation'], 0)
demog_proj_2025['age18_24'] = np.where((demog_proj_2025['age'] >= 18) & (demog_proj_2025['age'] <= 24), demog_proj_2025['totalPopulation'], 0)

In [26]:
demog_proj_2025 = demog_proj_2025.drop('age', axis = 1)

In [27]:
demog_proj_2025['fipsCode'] = demog_proj_2025['fipsCode'].astype(str)
demog_proj_2025['year'] = demog_proj_2025['year'].astype(str)

In [28]:
demog_2025_aggr = demog_proj_2025.groupby('county', as_index=False).agg(sum)

In [29]:
demog_2025_aggr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 0 to 63
Data columns (total 24 columns):
county              64 non-null object
malePopulation      64 non-null float64
femalePopulation    64 non-null float64
totalPopulation     64 non-null float64
age85_pl            64 non-null float64
age80_84            64 non-null float64
age75_79            64 non-null float64
age70_74            64 non-null float64
age65_69            64 non-null float64
age60_64            64 non-null float64
age55_59            64 non-null float64
age50_54            64 non-null float64
age45_49            64 non-null float64
age40_44            64 non-null float64
age35_39            64 non-null float64
age30_34            64 non-null float64
age25_29            64 non-null float64
age20_24            64 non-null float64
age15_19            64 non-null float64
age10_14            64 non-null float64
age5_9              64 non-null float64
ageless_5           64 non-null float64
ageless_18        

In [30]:
demog_2025_aggr.columns = [col_name + '_2025' for col_name in demog_2025_aggr.columns]

In [31]:
print(demog_2025_aggr.loc[demog_2025_aggr['ageless_5_2025'] == 0])

   county_2025  malePopulation_2025  femalePopulation_2025  \
0        Adams               6448.0                 7493.0   
2     Arapahoe               5633.0                 5540.0   
6      Boulder               6641.0                 7859.0   
7   Broomfield              43844.0                43952.0   
16      Denver               5664.0                 7148.0   
18     Douglas               6048.0                 6519.0   
20     El Paso               5888.0                 6220.0   
30   Jefferson               6231.0                 6144.0   
35     Larimer               7726.0                 6407.0   
39        Mesa              45372.0                48536.0   
51      Pueblo              25376.0                23546.0   
62        Weld               8817.0                 8052.0   

    totalPopulation_2025  age85_pl_2025  age80_84_2025  age75_79_2025  \
0                 5589.0         5589.0            0.0            0.0   
2                 4909.0         4909.0        

In [32]:
demog_all_s1 = pd.merge(demog_2012, demog_2017, left_on = 'county_2012', right_on = 'county_2017')

In [33]:
demog_all_s2 = pd.merge(demog_all_s1, demog_2025_aggr, left_on = 'county_2012', right_on = 'county_2025')

In [34]:
demog_all = pd.merge(demog_all_s2, free_reduced_2017, left_on = 'county_2012', right_on = 'county_name_2017')

In [35]:
demog_all.sort_values(by=['county_2012'], inplace=True)

In [36]:
demog_all.head(20)

,pop_2012,county_2012,hispanic_2012,white_nh_2012,black_nh_2012,ntvam_nh_2012,asian_nh_2012,hawpi_nh_2012,other_nh_2012,twoplus_nh_2012,...,age5_9_2025,ageless_5_2025,ageless_18_2025,age18_24_2025,county_code_2017,county_name_2017,pk-12_count_2017,not_eligible_2017,free_and_reduced_2017,pcnt_free_and_reduced_2017
30,442996,Adams,167556,235991,12970,2104,15304,515,787,7769,...,0.0,0.0,0.0,0.0,01,Adams,83921.0,42544.0,41358.0,0.492821
14,15750,Alamosa,7185,7767,110,32,59,14,112,471,...,1123.0,1325.0,4767.0,2554.0,02,Alamosa,2636.0,983.0,1653.0,0.627086
0,574357,Arapahoe,105174,364766,55629,2211,28067,1166,1267,16077,...,0.0,0.0,0.0,0.0,03,Arapahoe,119630.0,68264.0,51366.0,0.429374
16,12109,Archuleta,2157,9493,9,33,117,9,0,291,...,703.0,834.0,2820.0,921.0,04,Archuleta,1568.0,846.0,722.0,0.460459
13,3783,Baca,347,3311,16,33,30,2,5,39,...,209.0,211.0,772.0,212.0,05,Baca,615.0,230.0,385.0,0.626016
17,6192,Bent,1908,3445,525,147,85,0,16,66,...,286.0,332.0,1068.0,537.0,06,Bent,1447.0,391.0,1056.0,0.729786
31,297218,Boulder,39475,235676,2230,816,11876,102,598,6445,...,0.0,0.0,0.0,0.0,07,Boulder,63360.0,46747.0,16613.0,0.262200
19,17879,Chaffee,1722,15439,500,71,60,0,0,87,...,1017.0,1134.0,3869.0,1728.0,08,Chaffee,2227.0,1477.0,750.0,0.336776
55,2095,Cheyenne,308,1761,5,1,11,0,0,9,...,140.0,168.0,522.0,131.0,09,Cheyenne,320.0,174.0,146.0,0.456250
33,9059,Clear Creek,453,8327,27,7,45,0,0,200,...,520.0,564.0,1958.0,689.0,10,Clear Creek,858.0,627.0,231.0,0.269231


In [37]:
file_n = '../../npsg_datafiles/demographics_all_years.csv'
demog_all.to_csv(file_n, index=False)